Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [10]:
from random import choices, randint, random, uniform
from math import exp

import lab9_lib

In [11]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    # here, we print the the genome (as a string) and the fitness function of this genome
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
    # fitness(ind) passes the genome using the __call__ function
print(fitness.calls)

01010101100010100111000010100000001110010111011111: 35.56%
11110010010110101000101000111111110111001001010111: 23.33%
10000011010010010101111010001101011110000010111100: 35.56%
00000001110101110000100110000101101011001010111001: 7.36%
10101010110111000011100000010011010110111101100011: 23.34%
00111001100111001010101001111011000010011110001010: 23.34%
00110100110000100111111001110001111000000001010110: 29.56%
11110101101100010111101011101110110001111000111101: 9.11%
11011000010010111111010001010010111110110101010000: 7.33%
11010011111000010011001010000111110110011110101011: 9.13%
10


## Local Search Algorithm
This Local Search utilizes the best individual obtained from the Genetic Algorithm. Initially, I set up the Local Search variables. During the algorithm's execution, neighbors are generated by randomly flipping a subset of bits, deviating from the known Simulated Annealing-based approach.

In [12]:
class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=10):
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness <= self.best_fitness:
                self.iterations_without_improvement += 1
            else:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0

            if self.best_fitness == 1.0:
                break

    def generate_neighbor(self, individual):
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        # randomly flipping of bits
        mutation_rate = 0.1  # mutation rate of 10%
        for index in index_to_mutate:
            if random() < mutation_rate:
                neighbor[index] = 1 - neighbor[index]

        return neighbor

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        delta_fitness = new_fitness - current_fitness
        if delta_fitness < 0:
            return 1.0
        else:
            return exp(-delta_fitness / temperature)

## Genetic Algorithm
Here, I define the true genetic algorithm.

In [13]:
class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5):
        # Setting up Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.max_generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def run(self):
        best_individual_after_ls = None
        count_no_improvement_steps = 0

        # Population initialization
        population = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        for generation in range(self.max_generations):
            # Fitness evaluation
            fitness_scores = [self.problem(individual) for individual in population]
            best_individual = population[fitness_scores.index(max(fitness_scores))]

            # Applying local search to the best individual
            local_search = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            local_search.run()
            best_individual_after_ls = local_search.best_individual
            fitness_after_local = local_search.best_fitness

            # Checking for fitness improvement after local search
            if fitness_after_local > self.problem(best_individual):
                count_no_improvement_steps = 0
            else:
                count_no_improvement_steps += 1

            # Termination condition based on consecutive generations with no improvement
            if count_no_improvement_steps >= self.convergence_threshold:
                break

            # Replacing an individual in the population if fitness improves after local search
            if fitness_after_local > self.problem(best_individual):
                index_to_replace = fitness_scores.index(min(fitness_scores))
                population[index_to_replace] = best_individual_after_ls

            # Updating population using crossover and mutation
            new_population = self.crossover(population)
            self.mutate(new_population)
            population = new_population

        # Returning the best individual after all generations
        return best_individual_after_ls

    def crossover(self, population):
        new_population = []

        for _ in range(self.population_size // 2):
            # Tournament selection of parents
            parent1 = choices(population)[0]
            parent2 = choices(population)[0]

            # Two-point crossover
            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
            child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

            new_population.extend([child1, child2])

        return new_population

    def mutate(self, population):
        for i in range(len(population)):
            for j in range(self.genome_size):
                if random() < self.mutation_rate:
                    population[i][j] = 1 - population[i][j]



Here there is the main part, in which I call the genetic algorithm and I print all the results.

In [15]:
# Usage of the Genetic Algorithm
instances = [1, 2, 5, 10]

for instance in instances:
    fitness_value = instance
    fitness = lab9_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Instance {instance}")
    print(f"Best solution: {''.join(map(str, best_solution))}")
    print(f"Best fitness value: {fitness(best_solution):.2%}")
    print(f"Total calls: {fitness.calls}\n")

Instance 1
Best solution: 11011111010100011000111000100011101000101111001000101110110111101100100101101010101100110110111001111001111100010101001110111001010101010101101011011001010110000101111101110100010101011110110110100111010011110110011111000011010000011111110111010110011011110001110100100110011101001111111100001111010110110101011100011000101001110011011000011000011111101010100110101110111110100100001001111100110000000111110011111110001111011010011011110011100101100010101010010101001001110010111111111011111011100111101101111111111010100001111010111111110011100110110100001110011011101100011111111011011110010101010001001010010101101001010010000000111110011101111101010101000111101011101011100110001100100000101011001001010111011010111111010110001110111001101001101011100000110010010011110110111110000000001110111111000110000100110100010100101011001011010101101100111000011001110100100110101010111101101010111101001100011110011101000000000110001110101000010011110100001001111000100110000000